## Building A Chatbot
---
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")


### LangSmith tracing only works when:
---

You're using LCEL (LangChain Expression Language) chains like:

* prompt | model | parser

* Or, you're calling invoke() on a full Runnable chain (not just ChatModel.invoke() directly).

* Calling model.invoke(...) alone is too low-level — it doesn’t generate a trace in LangSmith unless it's wrapped inside a Runnable chain.

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage

# invoking the model with a human message
# The model will respond with a message based on the input provided
# We can replace the content of the HumanMessage with any text you want to test
result = model.invoke([
    HumanMessage(content="Hi , My name is Irfan and I am currently learning GenAi")
])

result

In [ ]:
print(result.content)

In [ ]:
from langchain_core.messages import AIMessage

# using this we can create a conversation
# This is useful for testing how the model responds to different inputs
# we do so by passing a list of messages to the model
# thus our model remembers the context of the conversation
response = model.invoke(
    [
        HumanMessage(content="Hi , My name is Irfan and I am currently learning GenAi"),
        
        AIMessage(content="That's great, Irfan! \n\nIt's wonderful to hear you're diving into the world of Generative AI. It's a fascinating and rapidly evolving field with a lot of potential. \n\nWhat specifically are you interested in learning about GenAI?  \n\nPerhaps you'd like to know more about:\n\n* **Different types of generative models:** Like text-to-image, text-to-code, or music generation?\n* **How these models are trained:**  The role of large datasets and deep learning?\n* **Applications of GenAI:**  In fields like art, writing, programming, or research?\n* **Ethical considerations:**  Bias, fairness, and the impact of GenAI on society?\n\nTell me more about your goals, and I'll do my best to help you on your GenAI learning journey!\n"),
        
        HumanMessage(content="Hey, what's my name and what i am currently learning?")
    ]
)
print(response)

In [ ]:
print(response.content)

## Message History
---
- We can use a Message History class to wrap our model and make it stateful (so that it can remember the context w.r.t to any person talking with the model).
-  This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. !
### A session ID helps:
- Track user interaction over time (within a conversation or chat window).

- Maintain context between turns/messages in a conversation.

- Optionally store state, preferences, history, or memory (if memory is enabled).

### Note:
-  By default, one session ID does NOT remember another session's conversation — unless you explicitly store and manage the memory.

In [ ]:

# ChatMessageHistory  is a class that allows us to keep track of the messages exchanged in a chat session.
from langchain_community.chat_message_histories import ChatMessageHistory

# BaseChatMessageHistory is an abstract class that defines the interface for chat message histories.
# It provides methods for adding messages, retrieving messages, and clearing the history.
from langchain_core.chat_history import BaseChatMessageHistory

# RunnableWithMessageHistory is a class that wraps a model and allows it to maintain a message history.
# It provides methods for invoking the model with a message history and retrieving the message history.
from langchain_core.runnables.history import RunnableWithMessageHistory




# We will use a dictionary to store the session history for each session id
# This will allow us to keep track of multiple chat sessions simultaneously
# The session id will be used to distinguish one chat session from another
store={}




# This function retrieves the session history for a given session id.
# If the session id does not exist in the store, it creates a new ChatMessageHistory instance and adds it to the store.
# This way, we can ensure that each session has its own message history.

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        # If the session id does not exist, create a new ChatMessageHistory instance
        store[session_id]=ChatMessageHistory()
    return store[session_id]



# RunnableWithMessageHistory is used to wrap the model and provide it with the session history.
# This allows the model to maintain a message history for each session and respond accordingly.
with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
# it is used to invoke the model with a session id
# For now we are using a hardcoded session id
config={"configurable":{"session_id":"chat1"}}

### config: This contains the session ID and maybe other metadata to track the user's conversation history.

In [ ]:
# invoking the model with a human message and the session id
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Irfan and I am currently learning GenAi")],
    config=config
)

In [ ]:
print(response.content)

In [ ]:
# Now, let's ask the model a question that requires it to remember the context of the conversation.
# This will test if the model can recall the information provided earlier in the conversation.
# The model should be able to respond with the name and the topic of learning based on the previous messages.
result = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config, # this is the session id we are using to distinguish one chat session from another
)
result

In [ ]:
print(result.content)

In [ ]:
## changing the config --> means changing the session id

config1={"configurable":{"session_id":"chat2"}}

# now my model will not remember the previous conversation
# becase we are using a different session id
#  and this session id does not have any previous messages stored
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

In [ ]:
print(response.content)

In [ ]:
# # Now, let's invoke the model with a different session id
# This will create a new chat session and the model will not remember the previous conversation.
# This is useful for testing how the model behaves in different chat sessions.
# The model should respond with a message indicating that it does not remember the previous conversation.
# This is because we are using a different session id, which does not have any previous messages
# stored in the session history.
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

In [ ]:
# testing the model with a different session id
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

## Prompt templates
---
- Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. 

- Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [ ]:
# ChatPromptTemplate,MessagesPlaceholder help us to create a prompt template that can be used to format the input messages for the model.
# This allows us to define a system message and a placeholder for the human messages.
# The system message provides context to the model, and the MessagesPlaceholder allows us to dynamically insert human messages into the prompt.

from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the best of your ability"),

        # MessagesPlaceholder uses a variable name to hold the human messages
        # whatever human message we give its need to be in key value pairs, and the key name should be messages
        
        # MessagesPlaceholder is a placeholder for chat messages that lets you insert a list of messages (chat history) dynamically into a prompt template.
        MessagesPlaceholder(variable_name="messages")
    ]
)




#  we now have a prompt template that can be used to format the input messages for the model.
# This prompt template can be used to create a chain that combines the prompt with the model.
# The chain will take the human messages as input and format them using the prompt template before passing
# them to the model for generating a response.

chain = prompt | model

In [ ]:
# invoking the chain with a human message
# This will format the message using the prompt template and then pass it to the model for generating a response.
# The model will respond with a message based on the input provided
chain.invoke({"messages":[HumanMessage(content="Hi My name is irfan")]})

In [ ]:
# we are now using the RunnableWithMessageHistory to wrap the chain
# This allows us to maintain a message history for the chain and respond accordingly.

with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config = {"configurable": {"session_id": "chat3"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is irfan")],
    config=config
)

response

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

### Add more complexity

In [ ]:


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Irfan")],"language":"Hindi"})
response.content

### Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

### input_messages_key
This tells LangChain:

Which key in your input dict holds the list of messages ([HumanMessage, AIMessage, ...]) that should be passed to the MessagesPlaceholder in the prompt.

In [ ]:

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Irfan")],"language":"Hindi"},
    config=config
)
repsonse.content

In [ ]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [ ]:
response.content

## Managing the Conversation History
---
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

- 'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [ ]:
from langchain_core.messages import SystemMessage,trim_messages


trimmer = trim_messages(
    max_tokens=45,         # ✅ Max total tokens allowed across the selected messages

    strategy="last",       # ✅ Trimming strategy: 
                           # "last" means keep the most recent messages and trim older ones first

    token_counter=model,   # ✅ Token counter (usually the LLM object or tokenizer)
                           # Used to count how many tokens each message will consume

    include_system=True,   # ✅ Include the system message (if present) in trimming/calculation
                           # System prompts are often important for guiding behavior

    allow_partial=False,   # ✅ Don’t allow partial messages — either include a full message or not at all

    start_on="human"       # ✅ Start trimming/checking from the **last human message**
                           # Useful when you want to prioritize keeping the most recent human turns
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

In [ ]:
from operator import itemgetter  # Used to extract specific keys from a dictionary

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

#  i am not getting because the trimmer trimmed the context of icecream

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

In [ ]:
## Lets wrap this in the Message History


with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [ ]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content